In [2]:
import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px

w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 8, #nColumns
    wDimY = 128, #nRows
    xBatches = 50,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'binary'
)

wx = w @ x.T

[i for i in zip(wx.T,wx_outBits.T)]
wx.T[0], q.saturating_clip(wx.T[0],inBits = 6,outBits = 4)
# q.get_array_bits(res)
# px.histogram(x=res.flatten(),title=f'{res.mean()}')

(array([-5,  3,  1,  3, -8, -5, -5,  5]),
 array([-2,  0,  0,  0, -2, -2, -2,  1]))

In [7]:
wx_outBits.T[::-1].T

array([[ 1, -2, -2, -2,  0,  0,  0, -2],
       [-1, -1,  0, -3,  0,  0,  0,  0],
       [-1,  0,  1,  1, -1,  0, -2, -1],
       [-1, -3, -1,  0,  0, -1, -3, -1],
       [-1, -1, -3,  1, -2, -2, -4, -3],
       [ 0,  0, -2, -3, -2, -1, -4, -3],
       [-1, -2,  2, -1, -2, -1, -1,  1],
       [-1,  0,  1, -1,  1,  2,  0,  0],
       [ 0,  0,  3,  0,  0, -2,  2,  1],
       [-1,  3,  0,  1,  0,  2,  0, -1],
       [-1,  2,  2,  1, -3,  0, -1, -1],
       [ 0, -1, -3, -1,  1,  2, -1, -1],
       [-1,  0, -3,  0, -2, -2,  1, -3],
       [ 0,  0,  0,  2,  0, -1,  2,  2],
       [ 1,  0,  0,  0, -1,  0,  0, -2],
       [-1,  3,  3,  2,  2,  3,  2,  1],
       [ 1,  0,  1,  0,  1, -1,  0,  1],
       [-2,  1, -2,  0, -2, -1, -2,  0],
       [ 0, -2,  1,  0, -1, -2,  1, -1],
       [-1,  0,  0,  0,  0, -1,  1,  1],
       [ 0, -3, -3, -2,  0,  0,  1, -1],
       [-1,  1,  0,  0, -1, -1,  1, -2],
       [ 2,  1,  1, -1,  3,  3,  2,  3],
       [ 0,  1, -1,  2, -1,  0, -1, -2],
       [-1, -2, 

In [66]:
[np.dot(w[i],x[0]) for i in range(w.shape[0])]

[-5, 3, 1, 3, -8, -5, -5, 5]

In [64]:
(w @ x.T).T

array([[ -5,   3,   1,   3,  -8,  -5,  -5,   5],
       [  2,   3,   2,   3, -11,   1,  -2,  -3],
       [ -3,  -6,   0,  -3,   7,   4,   1,  -3],
       [ -2, -11,  -1,   3,   3,  -4, -12,  -2],
       [-10, -15,  -5,  -5,   5, -12,  -2,  -2],
       [ -9, -13,  -1,  -8, -10,  -7,   0,   1],
       [  7,  -2,  -2,  -8,  -4,   9,  -5,  -1],
       [  1,   2,   8,   5,  -4,   6,   2,  -1],
       [  6,   9,  -6,   2,   0,  13,   2,   0],
       [ -3,   3,  11,   1,   7,   0,  14,  -3],
       [ -2,  -2,   1,  -9,   5,   8,   9,  -2],
       [ -2,  -1,  10,   7,  -3,  -9,  -1,   3],
       [-12,   7,  -8,  -7,   0,  -9,   0,  -4],
       [  9,  11,  -1,   0,   8,   3,   0,   2],
       [ -7,   3,   1,  -4,   1,   1,   1,   6],
       [  6,   8,  13,  11,  10,  12,  13,  -3],
       [  6,   0,  -2,   6,   3,   6,   0,   5],
       [  1,  -6,  -1,  -5,   0,  -6,   5,  -6],
       [ -4,   5,  -5,  -1,   2,   7,  -5,   3],
       [  5,   7,  -4,   0,   1,   3,   0,  -2],
       [ -2,   5,   